# RDP over Reverse SSH Tunnel WFP
Detects svchost hosting RDP termsvcs communicating with the loopback address and on TCP port 3389

## Rule Content
```
- title: RDP over Reverse SSH Tunnel WFP
  id: 5bed80b6-b3e8-428e-a3ae-d3c757589e41
  status: experimental
  description: Detects svchost hosting RDP termsvcs communicating with the loopback
    address and on TCP port 3389
  references:
  - https://twitter.com/SBousseaden/status/1096148422984384514
  author: Samir Bousseaden
  date: 2019/02/16
  tags:
  - attack.defense_evasion
  - attack.command_and_control
  - attack.t1076
  - car.2013-07-002
  logsource:
    product: windows
    service: security
    category: null
  detection:
    selection:
      EventID: 5156
    sourceRDP:
      SourcePort: 3389
      DestinationAddress:
      - 127.*
      - ::1
    destinationRDP:
      DestinationPort: 3389
      SourceAddress:
      - 127.*
      - ::1
    condition: selection and ( sourceRDP or destinationRDP )
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"5156" AND ((src_port:"3389" AND DestinationAddress.keyword:(127.* OR \:\:1)) OR (dst_port:"3389" AND src_ip_addr.keyword:(127.* OR \:\:1))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()